In [2]:
!pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/a2/1a/37c7ee1bc806d6c32621fecc72c19f6a9f9b4369e5e8f406a7c16d49f031/folium-0.15.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/2f/e7/603b136221de923055716d23e3047da71f92e0d8ba2c4517ce49a54fe768/branca-0.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/100.3 kB ? eta -:--:--
   ---------------------------------------- 100.3/100.3 kB 2.9 MB/s eta 0:00:00


In [74]:
# 라이브러리 불러오기
import requests
import pandas as pd
import numpy as np
import folium # 지도 시각화 라이브러리
from folium.plugins import MiniMap

In [75]:
# 장소 데이터를 가져오는 함수
def elec_location(region, page_num):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    # region에는 원하는 검색어가 들어감
    params = {'query':region, 'page':page_num}
    # 카카오맵 API 키 입력
    headers = {"Authorization":"KakaoAK 976f96e6f22f30bb25f87793cc172f62"}

    places = requests.get(url, params=params, headers=headers).json()['documents']
    total = requests.get(url, params=params, headers=headers).json()['meta']['total_count']
    # 카카오맵 API는 최대 45개의 장소만을 가져올 수 있음
    if total > 45:
        print(total, '개 중 45개 데이터만을 가져왔습니다.')
    else:
        print("모든 데이터를 가져왔습니다.")
    return places


In [76]:
# elec_location()에서 얻은 결과값을 하나씩 분리하여 저장
def elec_info(places):
    x = []
    y = []
    stores = []
    road_address = []
    place_url = []
    ID = []
    for place in places:
        x.append(float(place['x'])) # 위도
        y.append(float(place['y'])) # 경도
        stores.append(place['place_name']) # 장소명
        road_address.append(place['road_address_name']) # 장소의 도로명주소
        place_url.append(place['place_url']) # 장소의 사이트 주소
        ID.append(place['id']) # 장소의 ID

    ar = np.array([ID, stores, x, y, road_address, place_url]).T 
    # 데이터프레임 형태로 위의 정보들을 저장
    df = pd.DataFrame(ar, columns=['ID', 'stores', 'x', 'y', 'road_address', 'place_url'])
    return df


In [77]:
# 여러 개의 키워드를 검색할 때 사용
def keywords(location_name):
    df = None
    for loca in location_name:
        # 1페이지 당 15개의 결과를 가짐. 총 3페이지의 정보만 불러올 수 있음
        for page in range(1,4):
            local_name = elec_location(loca, page)
            local_elec_info = elec_info(local_name)

            if df is None:
                df = local_elec_info
            elif local_elec_info is None:
                continue
            else:
                df = pd.concat([df, local_elec_info], join='outer', ignore_index=True)
    return df

In [78]:
# 지도를 표출하는 함수
def make_map(dfs):
    # 기준지 - 제주국제공항
    m = folium.Map(location=[33.5101562, 126.4861157], zoom_start=12)
    # 미니맵 추가
    minimap = MiniMap()
    m.add_child(minimap)
    # 마커 추가
    for i in range(len(dfs)):
        folium.Marker([df['y'][i], df['x'][i]],
                      tooltip=dfs['stores'][i],
                      popup=dfs['place_url'][i],).add_to(m)
    return m 

In [85]:
# 검색을 원하는 키워드를 리스트 형태로 입력
# 관광지
location = ['제주 오름', '제주 테마파크', '제주 식물원', '제주 박물관',
            '제주 해수욕장', '제주 산봉우리', '제주 해안도로']
df = keywords(location)
df = df.drop_duplicates(['ID']) # 중복 결과 제거
df = df.reset_index() # 인덱스 리셋

make_map(df) # 지도로 표시

685 개 중 45개 데이터만을 가져왔습니다.
685 개 중 45개 데이터만을 가져왔습니다.
685 개 중 45개 데이터만을 가져왔습니다.
195 개 중 45개 데이터만을 가져왔습니다.
195 개 중 45개 데이터만을 가져왔습니다.
195 개 중 45개 데이터만을 가져왔습니다.
50 개 중 45개 데이터만을 가져왔습니다.
50 개 중 45개 데이터만을 가져왔습니다.
50 개 중 45개 데이터만을 가져왔습니다.
130 개 중 45개 데이터만을 가져왔습니다.
130 개 중 45개 데이터만을 가져왔습니다.
130 개 중 45개 데이터만을 가져왔습니다.
120 개 중 45개 데이터만을 가져왔습니다.
120 개 중 45개 데이터만을 가져왔습니다.
120 개 중 45개 데이터만을 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
56 개 중 45개 데이터만을 가져왔습니다.
56 개 중 45개 데이터만을 가져왔습니다.
56 개 중 45개 데이터만을 가져왔습니다.


In [86]:
# df의 데이터 확인 및 경도 값을 기준으로 정렬
df_sorted = df.sort_values(by=['road_address'])
display(df_sorted.head())

,index,ID,stores,x,y,road_address,place_url
0,0,10626439,용눈이오름,126.83268298189506,33.45988023593872,,http://place.map.kakao.com/10626439
241,241,17475343,월랑봉,126.82164625375279,33.47582330061832,,http://place.map.kakao.com/17475343
240,240,18742637,삼의봉,126.561924426462,33.4402458803769,,http://place.map.kakao.com/18742637
121,121,18039453,방림원 백화동산,126.269190368926,33.337033331953,,http://place.map.kakao.com/18039453
239,239,24022942,삼각봉,126.52655010411168,33.3738000688851,,http://place.map.kakao.com/24022942


In [87]:
# 데이터를 csv 파일로 저장
df_sorted.to_csv("tour_region.csv", index=False, encoding="cp949")

In [82]:
# 검색을 원하는 키워드를 리스트 형태로 입력
# 카페, 기념품
location = ['제주 카페', '제주 디저트', '제주 기념품']

df = keywords(location)
df = df.drop_duplicates(['ID']) # 중복 결과 제거
df = df.reset_index() # 인덱스 리셋

make_map(df) # 지도로 표시

4244 개 중 45개 데이터만을 가져왔습니다.
4244 개 중 45개 데이터만을 가져왔습니다.
4244 개 중 45개 데이터만을 가져왔습니다.
676 개 중 45개 데이터만을 가져왔습니다.
676 개 중 45개 데이터만을 가져왔습니다.
676 개 중 45개 데이터만을 가져왔습니다.
263 개 중 45개 데이터만을 가져왔습니다.
263 개 중 45개 데이터만을 가져왔습니다.
263 개 중 45개 데이터만을 가져왔습니다.


In [83]:
# df의 데이터 확인 및 주소를 기준으로 정렬
df_sorted = df.sort_values(by=['road_address'])
display(df_sorted.head())

,index,ID,stores,x,y,road_address,place_url
40,40,1028406499,꽃이피다앤블라썸,126.55466214000168,33.24292105533639,제주특별자치도 서귀포시 남성로 136,http://place.map.kakao.com/1028406499
19,19,19639423,카페서연의집,126.65558061062826,33.26940684756492,제주특별자치도 서귀포시 남원읍 위미해안로 86,http://place.map.kakao.com/19639423
14,14,28741599,미쁜제과,126.180059640842,33.2691601160601,제주특별자치도 서귀포시 대정읍 도원남로 16,http://place.map.kakao.com/28741599
64,74,27167127,재클린디저트 모슬포점,126.252000762497,33.2181498204647,제주특별자치도 서귀포시 대정읍 최남단해안로 26-1,http://place.map.kakao.com/27167127
11,11,1983315941,바다다,126.436715016074,33.2369068490562,제주특별자치도 서귀포시 대포로 148-15,http://place.map.kakao.com/1983315941


In [84]:
# 데이터를 csv 파일로 저장
df_sorted.to_csv("tour_cafe_souvenir.csv", index=False, encoding="cp949")

In [91]:
# 검색을 원하는 키워드를 리스트 형태로 입력
# 음식점
location = ['제주도 식당', '제주도 음식점', '제주도 맛집', '제주도 밥집',
           '애월읍 식당', '한림읍 식당', '서귀포시 식당', '제주공항 식당',
           '구좌읍 식당', '조천읍 식당', '표선읍 식당', '우도면 식당',
            '성산읍 식당', '한경면 식당']

df = keywords(location)
df = df.drop_duplicates(['ID']) # 중복 결과 제거
df = df.reset_index() # 인덱스 리셋

make_map(df) # 지도로 표시

18744 개 중 45개 데이터만을 가져왔습니다.
18744 개 중 45개 데이터만을 가져왔습니다.
18744 개 중 45개 데이터만을 가져왔습니다.
18744 개 중 45개 데이터만을 가져왔습니다.
18744 개 중 45개 데이터만을 가져왔습니다.
18744 개 중 45개 데이터만을 가져왔습니다.
18685 개 중 45개 데이터만을 가져왔습니다.
18685 개 중 45개 데이터만을 가져왔습니다.
18685 개 중 45개 데이터만을 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
1816 개 중 45개 데이터만을 가져왔습니다.
1816 개 중 45개 데이터만을 가져왔습니다.
1816 개 중 45개 데이터만을 가져왔습니다.
1380 개 중 45개 데이터만을 가져왔습니다.
1380 개 중 45개 데이터만을 가져왔습니다.
1380 개 중 45개 데이터만을 가져왔습니다.
5993 개 중 45개 데이터만을 가져왔습니다.
5993 개 중 45개 데이터만을 가져왔습니다.
5993 개 중 45개 데이터만을 가져왔습니다.
3033 개 중 45개 데이터만을 가져왔습니다.
3033 개 중 45개 데이터만을 가져왔습니다.
3033 개 중 45개 데이터만을 가져왔습니다.
1279 개 중 45개 데이터만을 가져왔습니다.
1279 개 중 45개 데이터만을 가져왔습니다.
1279 개 중 45개 데이터만을 가져왔습니다.
2719 개 중 45개 데이터만을 가져왔습니다.
2719 개 중 45개 데이터만을 가져왔습니다.
2719 개 중 45개 데이터만을 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
모든 데이터를 가져왔습니다.
1116 개 중 45개 데이터만을 가져왔습니다.
1116 개 중 45개 데이터만을 가져왔습니다.
1116 개 중 45개 데이터만을 가져왔습니다.
1089 개 중 45개 데이터만을 가져왔습니다.
1089 개 중 45개 데이터만을 가져왔습니다.
1089 개 중 45개 데이터만을 가져왔습니다.
770 

In [92]:
# df의 데이터 확인 및 주소를 기준으로 정렬
df_sorted = df.sort_values(by=['road_address'])
display(df_sorted.head())

,index,ID,stores,x,y,road_address,place_url
233,350,376472561,엔제리너스 제주공항국내선탑승점,126.492743164674,33.5070806552029,,http://place.map.kakao.com/376472561
112,208,18468300,닻,126.39207943939813,33.486053434042375,,http://place.map.kakao.com/18468300
400,524,1133384427,호랑호랑카페,126.92163924055625,33.449586326551795,,http://place.map.kakao.com/1133384427
204,318,483777957,파리바게뜨 PB제주공항렌터카하우스,126.49194991455421,33.504697208252075,,http://place.map.kakao.com/483777957
181,295,24839178,은희네해장국 서귀포점,126.5821866980296,33.26047393787188,제주특별자치도 서귀포시 516로 84,http://place.map.kakao.com/24839178


In [93]:
# 데이터를 csv 파일로 저장
df_sorted.to_csv("tour_restaurant.csv", index=False, encoding="cp949")